In [1]:
from keras_facenet import FaceNet
from mtcnn.mtcnn import MTCNN
from sklearn.metrics.pairwise import cosine_similarity
import cv2
from PIL import Image
import numpy as np

In [2]:
with open(r'C:\Users\Raum\Desktop\jec\code\experimental\array_of_frame\gg.npy', 'rb') as f:
    a = np.load(f)
a.shape

(1967,)

In [3]:
embedder = FaceNet()
detector = MTCNN()

In [4]:
def extraction_keras_lock(*file_name,show=None):
  if type(file_name) is tuple:
    file_name = [*file_name,]
  result = []
  for i in file_name:
    image = Image.open(i).convert('RGB')
    im_arr = np.array(image)
    N=0
    faces = detector.detect_faces(im_arr)
    x1,y1,width,height = faces[N]['box'] # faces[N]['box']
    x2,y2 = (x1+width),(y1+height)
    face = im_arr[y1:y2,x1:x2] # ตัดเอาเฉพาะใบหน้า
    image_face = Image.fromarray(face)
    image_face= image_face.resize((224,224))
    face_img = np.expand_dims(image_face,axis=0)
    result.append(embedder.embeddings(face_img))

  return result

In [5]:
def extraction_keras_all_cv(im_arr):
    position_crop = []
    embedding_faces = []
    position = []
    faces = detector.detect_faces(im_arr)
    
    if faces == []:
        return [],[]

    for i in faces:
        x1,y1,width,height = i['box']
        x2,y2 = (x1+width),(y1+height)
        position_crop.append([y1,y2,x1,x2])
        position.append([x1,y1,x2,y2])

    # print(len(position_crop))
    for i in position_crop:
        # print(i)
        face = im_arr[i[0]:i[1],i[2]:i[3]]
        image_face = cv2.resize(face,(224,224))
        faces_crops = np.expand_dims(image_face,axis=0)
        embedding_faces.append(embedder.embeddings(faces_crops))
        # print(embedding_faces)
    return embedding_faces,position

In [6]:
image=cv2.imread(r'C:\Users\Raum\Desktop\jec\code\dataface\pitha.jpg')
im_arr = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
embedding_faces,position = extraction_keras_all_cv(im_arr)
xx = extraction_keras_lock(r'C:\Users\Raum\Desktop\jec\code\dataface\Pita.jpg',r'C:\Users\Raum\Desktop\jec\code\dataface\siri.jpg')

In [21]:

log_cosine_similarity = []

Detection_check = np.zeros((len(position)))
for j in range(len(xx)):

    for i in range(len(position)):
        similar = cosine_similarity(xx[j], embedding_faces[i])
        log_cosine_similarity.append(similar)
        if similar>0.55 and Detection_check[i] ==0 :
            Detection_check[i] = 1
            break

for i in range(len(position)):
    if Detection_check[i] == 1:
          pass
    else :
        censor_region = (position[i][0],position[i][1],position[i][2],position[i][3])

        censored_area = image[censor_region[1]:censor_region[3], censor_region[0]:censor_region[2]]

        censored_width, censored_height = censored_area.shape[1], censored_area.shape[0]


        pixel_size = 12
        censored_area = cv2.resize(censored_area, (censored_width // pixel_size, censored_height // pixel_size))
        censored_area = cv2.resize(censored_area, (censored_width, censored_height), interpolation=cv2.INTER_NEAREST)

        image[censor_region[1]:censor_region[3], censor_region[0]:censor_region[2]] = censored_area



[array([[0.7874079]], dtype=float32),
 array([[0.2973188]], dtype=float32),
 array([[-0.03770198]], dtype=float32),
 array([[0.08064354]], dtype=float32),
 array([[0.27171814]], dtype=float32),
 array([[-0.05674545]], dtype=float32),
 array([[0.0596335]], dtype=float32),
 array([[0.05591889]], dtype=float32),
 array([[0.20720986]], dtype=float32),
 array([[0.06897811]], dtype=float32),
 array([[0.7230322]], dtype=float32)]

In [22]:
log_cosine_similarity

[array([[0.7874079]], dtype=float32),
 array([[0.2973188]], dtype=float32),
 array([[-0.03770198]], dtype=float32),
 array([[0.08064354]], dtype=float32),
 array([[0.27171814]], dtype=float32),
 array([[-0.05674545]], dtype=float32),
 array([[0.0596335]], dtype=float32),
 array([[0.05591889]], dtype=float32),
 array([[0.20720986]], dtype=float32),
 array([[0.06897811]], dtype=float32),
 array([[0.7230322]], dtype=float32)]

In [23]:
cv2.imwrite('ggggg.jpg',image)

True